In [1]:
# first run installations and data download
! pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file
! mkdir -p './storage/commits'


zsh:1: command not found: pip


In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [3]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/commits"
    )
    commit_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [10]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [11]:
if not index_loaded:
    # load data
    commit_docs = SimpleDirectoryReader(
        input_files=["./0001-adding-a-better-calculator.patch"]
    ).load_data()

    # build index
    vector_store_commit = MilvusVectorStore(dim=1536, collection_name="commit", overwrite=True)
    
    storage_context_commit = StorageContext.from_defaults(vector_store=vector_store_commit)
    commit_index = VectorStoreIndex.from_documents(commit_docs, storage_context=storage_context_commit)


    # persist index
    commit_index.storage_context.persist(persist_dir="./storage/commit")


In [12]:
commit_engine = commit_index.as_query_engine(similarity_top_k=3)

In [13]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=commit_engine,
        metadata=ToolMetadata(
            name="commit_summary",
            description=(
                "Provides summary information about a commit patch.  "
                "Need to manually run 'git format-patch -1 to generate the patch file."
            ),
        ),
    ),
]

In [14]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [15]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [18]:
response = agent.chat("What changes were made in this patch?")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: commit_summary
Action Input: {'input': '0001-adding-a-better-calculator.patch'}
Observation: The patch file "0001-adding-a-better-calculator.patch" introduces a new Jupyter notebook file named "better_calculator_agent.ipynb" that contains code for a better calculator agent. The notebook includes functions for multiplication, addition, and squaring integers. It also initializes an agent with these tools and an OpenAI model. The agent is capable of performing step-by-step calculations based on user input, demonstrating the order of operations and providing the results interactively.
Thought: I can answer without using any more tools.
Answer: The patch file "0001-adding-a-better-calculator.patch" introduces a new Jupyter notebook file named "better_calculator_agent.ipynb" that contains code for a better calculator agent. The notebook includes functions for multiplication, addition, and squaring integers. It also initial